In [1]:
#imports
#Main Library (Actual Neural Network Part)
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models
import neural_structured_learning as nsl

# #Helper Libraries (in order to interpret and view the data)
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Prints if a GPU is detected by the TensorFlow system
print(len(tf.config.list_physical_devices('GPU')) > 0)

True


In [3]:
#importing the MNIST dataset from tensorflow
from tensorflow.keras.datasets import cifar10 
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

In [4]:
# print(X_train[0].shape)
X_train = X_train / 255.0
X_test = X_test / 255.0

In [5]:
# #validating the data (making sure this is the data I want)
# fig, ax = plt.subplots(2,5, figsize = (5,5))
# ax = ax.flatten()
# for i in range(10):
#     im_idx = np.argwhere(Y_train == i)[0][0]
#     plottable_image = X_train[im_idx]
#     ax[i].imshow(plottable_image, interpolation='none')

In [6]:
#creating the model - Sequential 
model = models.Sequential()
#first conv. layer and pooling layer
model.add(layers.Conv2D(8, (2, 2), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
#second conv. layer and pooling layer
model.add(layers.Conv2D(16, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
#last conv. layer
model.add(layers.Conv2D(32, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
#last conv. layer
model.add(layers.Conv2D(64, (2, 2), activation='relu'))

In [7]:
#checking the output shape of the last conv. layer, so I can set the last Dense layer correctly
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 31, 31, 8)         104       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 8)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 16)        528       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 6, 32)          2080      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 32)         0

In [8]:
#Flattening layer will create vectors in order for the neural netowrk to actually make predictions
model.add(layers.Flatten())
#adding the last dense layer to allow the neural network to classify the images
model.add(layers.Dense(64, activation = 'relu'))
#classification layer
model.add(layers.Dense(10, activation = "softmax"))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 31, 31, 8)         104       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 8)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 16)        528       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 6, 32)          2080      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 32)         0

In [10]:
adv_config = nsl.configs.make_adv_reg_config(multiplier=0.2, adv_step_size=0.05)
adv_model = nsl.keras.AdversarialRegularization(model, adv_config = adv_config)

In [11]:
#compiler which configures the model
adv_model.compile(optimizer='adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics = ['accuracy'])

In [12]:
#training the model
history = adv_model.fit({'feature': X_train, 'label': Y_train}, batch_size = 10, epochs = 20, verbose = 1,
                    shuffle = True)

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
5000/5000 [==============================] - 108s 16ms/step - loss: 1.8897 - sparse_categorical_crossentropy: 1.5577 - sparse_categorical_accuracy: 0.4301 - scaled_adversarial_loss: 0.3320
Epoch 2/20
5000/5000 [==============================] - 80s 16ms/step - loss: 1.5683 - sparse_categorical_crossentropy: 1.2809 - sparse_categorical_accuracy: 0.5382 - scaled_adversarial_loss: 0.2874
Epoch 3/20
5000/5000 [==============================] - 80s 16ms/step - loss: 1.4407 - sparse_categorical_crossentropy: 1.1701 - sparse_categorical_accuracy: 0.5804 - scaled_adversarial_loss: 0.2705
Epoch 4/20
5000/5000 [====================

In [13]:
adv_model.evaluate({'feature': X_test, 'label': Y_test})

313/313 [==============================] - 6s 15ms/step - loss: 1.2749 - sparse_categorical_crossentropy: 1.0027 - sparse_categorical_accuracy: 0.6622 - scaled_adversarial_loss: 0.2721


[1.2748643159866333,
 1.0027482509613037,
 0.6621999740600586,
 0.2721160650253296]

In [ ]:
adv_model.save('complete_saved_adv_cifar10_model')

In [5]:
#loading the model
base_model = tf.keras.models.load_model('complete_saved_cifar10_model')

In [14]:
#Creating the adversarial attack for adversarial training
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

def create_adv(input_image, input_label, model_type = "adv_model"):
  with tf.GradientTape() as tape:
    tape.watch(input_image)
    if(model_type == "model"):
      prediction = base_model(input_image)
    else:
      prediction = model(input_image)
    loss = loss_object(input_label, prediction)

  gradient = tape.gradient(loss, input_image)
  signed_grad = tf.sign(gradient)
  return signed_grad

def fgsm(input_image, input_label, eps=0.25, model_type = "adv_model"):
  perturbation = create_adv(input_image, input_label, model_type)
  adv_image = input_image + perturbation * eps
  adv_image = tf.clip_by_value(adv_image, 0, 1)

  return adv_image

def pgd(input_image, input_label, num_steps=100, eps=0.25, alpha=0.01, model_type = "adv_model"):
  adv_image = input_image
  for i in range(num_steps):
    adv_image = fgsm(adv_image, input_label, alpha, model_type)
    perturbation = adv_image - input_image
    perturbation = tf.clip_by_value(perturbation, -eps, eps)
    adv_image = input_image + perturbation
  
  return adv_image


In [18]:
adv_x = fgsm(tf.convert_to_tensor(X_test), Y_test)
adv_model.evaluate({'feature': adv_x, 'label': Y_test})

313/313 [==============================] - 5s 17ms/step - loss: 8.5975 - sparse_categorical_crossentropy: 7.0372 - sparse_categorical_accuracy: 0.0664 - scaled_adversarial_loss: 1.5603


[8.597528457641602, 7.037239074707031, 0.06639999896287918, 1.5602933168411255]